In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import argparse
import os
import torch
import random
import numpy as np
import pandas as pd

In [4]:
os.chdir('/content/drive/MyDrive/work_space/Project/LTSF-Linear')
from exp.exp_main import Exp_Main

In [5]:
df = pd.read_csv('./data/weather_electric_data_2.csv')
df['date'] = pd.to_datetime(df['date'])
df

,date,서울_기온(°C),서울_누적강수량(mm),서울_풍향(deg),서울_풍속(m/s),서울_현지기압(hPa),서울_해면기압(hPa),서울_습도(%),서울_일사(MJ/m^2),서울_일조(Sec),...,천안_일조(Sec),공급능력(MW),현재수요(MW),공급예비력(MW),운영예비력(MW),요일,is_holiday_or_weekend,요일1,요일2,요일3
0,2012-06-01 00:05:00,18.3,0.0,21.4,1.4,1006.4,1016.4,70.7,0.00,0.0,...,0.0,64355.0,51342.53,13012.47,13012.47,4,0,1,0,1
1,2012-06-01 00:10:00,18.0,0.0,11.6,1.0,1006.3,1016.3,72.1,0.00,0.0,...,0.0,64355.0,51583.17,12771.83,12771.83,4,0,1,0,1
2,2012-06-01 00:15:00,17.9,0.0,6.2,1.2,1006.2,1016.2,72.7,0.00,0.0,...,0.0,64355.0,51631.69,12723.31,12723.31,4,0,1,0,1
3,2012-06-01 00:20:00,17.9,0.0,20.1,1.0,1006.1,1016.1,72.8,0.00,0.0,...,0.0,64355.0,51252.66,13102.35,13102.35,4,0,1,0,1
4,2012-06-01 00:25:00,17.8,0.0,18.0,0.8,1006.1,1016.2,72.8,0.00,0.0,...,0.0,64355.0,50888.38,13466.62,13466.62,4,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135866,2023-03-19 23:35:00,6.4,0.0,254.8,2.4,1007.8,1018.3,70.4,21.25,39900.0,...,39384.0,83627.7,58627.20,25000.50,10477.00,6,1,1,1,1
1135867,2023-03-19 23:40:00,6.4,0.0,250.4,2.2,1007.8,1018.3,70.5,21.25,39900.0,...,39384.0,83622.0,57941.20,25680.70,10712.70,6,1,1,1,1
1135868,2023-03-19 23:45:00,6.3,0.0,244.6,2.1,1007.7,1018.2,70.1,21.25,39900.0,...,39384.0,83618.9,57726.20,25892.70,10759.20,6,1,1,1,1
1135869,2023-03-19 23:50:00,6.3,0.0,245.7,2.1,1007.7,1018.2,68.7,21.25,39900.0,...,39384.0,83630.3,57755.40,25874.90,10628.70,6,1,1,1,1


In [9]:
start_time = pd.to_datetime('2023-03-13 00:00:00')
end_time = pd.to_datetime('2023-03-13 05:55:00')
filtered_df = df[(df['date'] >= start_time) & (df['date'] <= end_time)]
filtered_df

,date,서울_기온(°C),서울_누적강수량(mm),서울_풍향(deg),서울_풍속(m/s),서울_현지기압(hPa),서울_해면기압(hPa),서울_습도(%),서울_일사(MJ/m^2),서울_일조(Sec),...,천안_일조(Sec),공급능력(MW),현재수요(MW),공급예비력(MW),운영예비력(MW),요일,is_holiday_or_weekend,요일1,요일2,요일3
1133855,2023-03-13 00:00:00,0.6,5.6,298.1,5.2,1012.5,1023.3,39.8,3.68,2340.0,...,2160.0,84908.5,58782.0,26126.5,11493.2,0,0,0,0,1
1133856,2023-03-13 00:05:00,0.5,0.0,257.4,5.1,1012.5,1023.3,40.0,0.00,0.0,...,0.0,85917.5,58954.4,26963.1,11608.4,0,0,0,0,1
1133857,2023-03-13 00:10:00,0.5,0.0,274.3,4.9,1012.5,1023.3,39.7,0.00,0.0,...,0.0,86266.2,58710.6,27555.6,11721.4,0,0,0,0,1
1133858,2023-03-13 00:15:00,0.4,0.0,274.0,4.2,1012.4,1023.2,39.3,0.00,0.0,...,0.0,86280.3,58298.4,27981.8,11486.2,0,0,0,0,1
1133859,2023-03-13 00:20:00,0.4,0.0,276.6,5.8,1012.5,1023.3,39.3,0.00,0.0,...,0.0,86249.0,58263.7,27985.3,11020.6,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133922,2023-03-13 05:35:00,-1.7,0.0,267.4,2.1,1013.9,1024.8,38.7,0.00,0.0,...,0.0,89332.6,57783.8,29506.0,11837.6,0,0,0,0,1
1133923,2023-03-13 05:40:00,-1.8,0.0,263.0,2.9,1014.0,1024.9,39.3,0.00,0.0,...,0.0,89347.7,57690.0,29614.6,12074.7,0,0,0,0,1
1133924,2023-03-13 05:45:00,-1.8,0.0,260.1,3.0,1014.0,1024.9,40.7,0.00,0.0,...,0.0,89363.8,58030.6,29289.0,12014.5,0,0,0,0,1
1133925,2023-03-13 05:50:00,-1.7,0.0,261.8,3.6,1014.1,1025.0,39.8,0.00,0.0,...,0.0,89380.3,58570.3,28766.6,11684.8,0,0,0,0,1


In [10]:
filtered_df['현재수요(MW)']

1133855    58782.0
1133856    58954.4
1133857    58710.6
1133858    58298.4
1133859    58263.7
            ...   
1133922    57783.8
1133923    57690.0
1133924    58030.6
1133925    58570.3
1133926    58788.2
Name: 현재수요(MW), Length: 72, dtype: float64

In [14]:
real_df = pd.read_csv('/content/drive/MyDrive/work_space/Project/LTSF-Linear/results/D_Linear_Test_1_DLinear_custom_ftM_sl864_ll72_pl72_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0/real_prediction.csv')

In [15]:
real_df['현재수요(MW)']

0     61248.386719
1     59829.007812
2     58117.308594
3     61498.316406
4     59281.101562
          ...     
67    61546.812500
68    52406.000000
69    52526.179688
70    53310.722656
71    52951.500000
Name: 현재수요(MW), Length: 72, dtype: float64

In [18]:
(np.array(filtered_df['현재수요(MW)']) - np.array(real_df['현재수요(MW)'])).sum() / 72

-1438.7717773437496

In [30]:
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [17]:
torch.cuda.is_available()

True

In [25]:
temp_df = pd.read_csv('./data/temp_df.csv')
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108576 entries, 0 to 108575
Columns: 154 entries, date to 요일3
dtypes: float64(148), int64(5), object(1)
memory usage: 127.6+ MB


In [21]:
temp_df = pd.read_csv('./data/temp_df.csv', parse_dates=['date'])
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108576 entries, 0 to 108575
Columns: 154 entries, date to 요일3
dtypes: datetime64[ns](1), float64(148), int64(5)
memory usage: 127.6 MB


In [22]:
type(temp_df.date[0])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
weather_electric_data_2.csv

In [31]:
args = argparse.Namespace(
    # basic config
    is_training = 1,                                  # default = 1
    train_only = False,                               # default = False
    model_id = 'D_Linear_Test_1',
    model = 'DLinear',                                # options: [DLinear, NLinear, Autoformer, Informer, Transformer, etc...]

    # Data Loader
    data = 'custom',                                  # I dont know ... Maybe Name?
    root_path = './data/',                         #### Data Folder Path
    data_path = 'temp_df.csv',      #### Data File Path
    features = 'M',                                #### Forecating task, options: [M, S, MS], M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate.
    target ='현재수요(MW)',                          #### target feature in S or MS task. Its maybe name of target Columns

    # Need to check 'freq' works well.
    freq = '5min',                                 #### freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h.
    checkpoints = './checkpoints/',                   # location of model checkpoints (Need it?)

    # Forecating Task
    seq_len = (288*3),                                  #### Input Sequence Length.
    label_len = 72,                                #### Start Token Length.
    pred_len = 72,                                 #### Prediction Sequence Length.

    # DLinear
    individual = False,                               # DLinear: a linear layer for each variate(channel) individually (I dont know)

    # Formers
    embed_type = 0,                                   # 0: default 1: value embedding + temporal embedding + positional embedding 2: value embedding + temporal embedding 3: value embedding + positional embedding 4: value embedding.
    enc_in = 153,                                 #### Encoder Input Size. (I dont know why we need to control this?)
    dec_in = 7,                                       # Decoder Input Size.
    c_out = 7,                                        # Output size.
    d_model = 512,                                    # Dimension of model(size of model)
    n_heads = 8,                                      # Num of attention heads.
    e_layers = 2,                                     # Num of Encoder layers.
    d_layers = 1,                                     # Num of Decoder layers.
    d_ff = 2048,                                      # Dimension of fcn(Feed-Forward Network?)
    moving_avg = 25,                                  # Windoe size of moving average.
    factor = 1,                                       # Attention Factor.
    distil = True,                                    # Whether to use distilling in encoder, using this argument means not using distilling.
    dropout = 0.05,                                   # Dropout rate.
    embed = 'timeF',                                  # Time features encoding, options:[timeF, fixed, learned]
    activation = 'gelu',                              # Activation Function.
    output_attention = False,                         # Whether to output attention in ecoder.
    do_predict = True,                                # Whether to predict unseen future data.

    # Optimization
    num_workers = 10,                             ##### data loader num workers
    itr = 1,                                      ##### experiments times
    train_epochs = 1,                            ##### train epochs
    batch_size = 16,                              ##### batch size of train input data
    patience = 3,                                     # early stopping patience
    learning_rate = 0.01,                       ##### optimizer learning rate
    des = 'Exp',                                  ##### exp description (Almost 'Exp', maybe we dont need to change it)
    loss = 'mse',                                     # loss function
    lradj = 'type1',                                  # adjust learning rate
    use_amp = False,                                  # use automatic mixed precision training

    # GPU Setting
    use_gpu = True,                                   # Use GPU?
    gpu = 0,                                          # GPU
    use_multi_gpu = False,                            # Use Multiple GPUs? (r u reach guy?)
    devices = '0,1,2,3',                              # Device Indexs of multile GPUs.
    test_flop = False                                 # See utils/tools for usage ... I dont wanna check
)

In [27]:
# Arguments Check

args

Namespace(is_training=1, train_only=False, model_id='D_Linear_Test_1', model='DLinear', data='custom', root_path='./data/', data_path='temp_df.csv', features='M', target='현재수요(MW)', freq='5min', checkpoints='./checkpoints/', seq_len=864, label_len=72, pred_len=72, individual=False, embed_type=0, enc_in=153, dec_in=7, c_out=7, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=1, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=True, num_workers=10, itr=1, train_epochs=1, batch_size=16, patience=3, learning_rate=0.01, des='Exp', loss='mse', lradj='type1', use_amp=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', test_flop=False)

In [32]:
# Model Run.

args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.dvices = args.devices.replace(' ', '')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

Exp = Exp_Main

if args.is_training:
    for ii in range(args.itr):
        # setting record of experiments
        setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
            args.model_id,
            args.model,
            args.data,
            args.features,
            args.seq_len,
            args.label_len,
            args.pred_len,
            args.d_model,
            args.n_heads,
            args.e_layers,
            args.d_layers,
            args.d_ff,
            args.factor,
            args.embed,
            args.distil,
            args.des, ii)

        exp = Exp(args)  # set experiments
        print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        exp.train(setting)

        if not args.train_only:
            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.test(setting)

        if args.do_predict:
            print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            exp.predict(setting, True)

        torch.cuda.empty_cache()
else:
    ii = 0
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(args.model_id,
                                                                                                  args.model,
                                                                                                  args.data,
                                                                                                  args.features,
                                                                                                  args.seq_len,
                                                                                                  args.label_len,
                                                                                                  args.pred_len,
                                                                                                  args.d_model,
                                                                                                  args.n_heads,
                                                                                                  args.e_layers,
                                                                                                  args.d_layers,
                                                                                                  args.d_ff,
                                                                                                  args.factor,
                                                                                                  args.embed,
                                                                                                  args.distil,
                                                                                                  args.des, ii)

    exp = Exp(args)  # set experiments

    if args.do_predict:
        print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.predict(setting, True)
    else:
        print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.test(setting, test=1)
    torch.cuda.empty_cache()


Args in experiment:
Namespace(is_training=1, train_only=False, model_id='D_Linear_Test_1', model='DLinear', data='custom', root_path='./data/', data_path='temp_df.csv', features='M', target='현재수요(MW)', freq='5min', checkpoints='./checkpoints/', seq_len=864, label_len=72, pred_len=72, individual=False, embed_type=0, enc_in=153, dec_in=7, c_out=7, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=1, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=True, num_workers=10, itr=1, train_epochs=1, batch_size=16, patience=3, learning_rate=0.01, des='Exp', loss='mse', lradj='type1', use_amp=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', test_flop=False)
Use GPU: cuda:0
>>>>>>>start training : D_Linear_Test_1_DLinear_custom_ftM_sl864_ll72_pl72_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 75068
val 10787
test 21644
	iters: 100, epoch: 1 | loss: 0.3446701
	speed: 0